# Exercise 2 - NDVI Normalized Difference Vegetation Index
### The NDVI is computed as the difference between near-infrared (NIR) and red (RED) reflectance divided by their sum.
$$ NDVI = \frac{NIR-RED}{NIR+RED}$$


In [1]:

import ee
import folium
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()
# This is needed in python to add layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


Successfully saved authorization token.


In [6]:
#Load an image 
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318');
lat = 37.5010
lon = -122.1899
#Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5']);

ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']};
myMap = folium.Map(location=[lat, lon], zoom_start=10)
myMap.add_ee_layer(ndwi, ndwiViz,'NDWI');
myMap

##Landsat 8 NDWI, San Francisco bay area, USA. Cyan are low values, blue are high values.

In [9]:
# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4));


#Create visualization layers.
imageRGB = image.visualize(**{'bands': ['B5', 'B4', 'B3'], 'max': 0.5})
ndwiRGB = ndwiMasked.visualize(**{
  'min': 0.5,
  'max': 1,
  'palette': ['00FFFF', '0000FF']
})


mosaic = ee.ImageCollection([imageRGB, ndwiRGB]).mosaic()
map_mosaic = folium.Map(location=[37.5010, -122.1899], zoom_start=10)
map_mosaic.add_ee_layer(mosaic, None, 'mosaic')
map_mosaic